# 3. Test

In [1]:
import sys, os
import pandas as pd
sys.path.append("../../../../")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as influx_Client
from Clust.clust.ingestion.mongo.mongo_client import MongoClient
db_client = influx_Client.InfluxClient(ins.CLUSTDataServer2)
mongo_client = MongoClient(ins.CLUSTMetaInfo2)

import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cuda is available.


In [2]:
from Clust.clust.ML.common import ML_pipeline, tool

## 1. Data Preparation

In [3]:
app_name= "Hs2SwineFarmWithWeatherTime" # "Hs2SwineFarmWithWeatherTime", "energy"
model_method = 'GRU_rg'
model_clean = True
step = 'test'
bucket_name = 'integration'
data_clean_level=0

In [4]:
model_name = None
model_name = tool.get_default_model_name(model_name, app_name, model_method, model_clean)
collection_name = 'meta'
db_name = 'model'
model_meta_list= mongo_client.get_document_by_json(db_name, collection_name, {'modelName': model_name})
model_meta = model_meta_list[3] ## ??

In [5]:
model_meta

{'trainDataInfo': {'data_name': 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel0_trainX',
  'ingestion_param': {'start_time': '2021-02-01 00:00:00',
   'end_time': '2021-03-10 00:00:00',
   'ms_list_info': [['farm_swine_air', 'HS2'],
    ['weather_outdoor_keti_clean', 'sangju'],
    ['life_additional_Info', 'trigonometicInfoByHours']]},
  'integration_param': {'param': {},
   'method': 'meta',
   'integration_duration': 'common',
   'integration_frequency': 300},
  'clean_level': 0,
  'process_param': {'refine_param': {'removeDuplication': {'flag': False},
    'staticFrequency': {'flag': False, 'frequency': None}},
   'outlier_param': {'certainErrorToNaN': {'flag': False},
    'unCertainErrorToNaN': {'flag': False}},
   'imputation_param': {'flag': False}}},
 'modelName': 'Hs2SwineFarmWithWeatherTime_GRU_rg_True',
 'dataSplitMode': 'step_split',
 'featureXList': ['Temperature', 'out_temp', 'sin_hour'],
 'featureyList': ['Temperature'],
 'data_y_flag': False,
 'trainDataType': 'time

### 1-2. Data Ingestion
#### 1-2-1. X-y Data Ingestion pipeline

In [6]:
model_purpose = model_meta['modelPurpose']
dataset_name = model_purpose + '_' + app_name  
scaler_param= model_meta['scalerParam']
data_name_X = dataset_name + '_cleanLevel' + str(data_clean_level)+'_'+step+'X'
data_name_y = dataset_name+'_cleanLevel' + str(data_clean_level)+'_'+ step+'y'
data_y_flag = model_meta['data_y_flag']

In [7]:
feature_X_list = model_meta['featureXList']
feature_y_list = model_meta["featureyList"]
model_file_path = model_meta['files']['modelFile']["filePath"]

In [8]:
# Data Ingestion
ingestion_method = 'ms_all'
ingestion_param_X = {
    "bucket_name" : bucket_name,
    'ms_name' : data_name_X,
    'feature_list' : feature_X_list                              
}
ingestion_param_y = {
    "bucket_name" : bucket_name,
    'ms_name' : data_name_y,
    'feature_list' : feature_y_list                              
}

In [9]:
data_X, data_y = ML_pipeline.Xy_data_preparation(ingestion_param_X, data_y_flag, ingestion_param_y, ingestion_method, db_client)

#### 1-2-2. Random Nan Insert (Test)
#### 1-2-3. Data scaling

In [10]:
X_scaler_file_path = model_meta['files']['XScalerFile']["filePath"]
y_scaler_file_path = model_meta['files']['yScalerFile']["filePath"]
y_scaler_file_path

'./scaler/forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel0_trainy/minmax/2ef658edf359871df2a5d6f044a18200/scaler.pkl'

In [11]:
scaler_param = model_meta["scalerParam"]
from Clust.clust.ML.tool import scaler as ml_scaler
def Xy_data_scaling_test(data_X, data_y, scaler_file_path, scaler_param):
    test_X, scaler_X = ml_scaler.get_scaled_test_data(data_X, X_scaler_file_path, scaler_param)
    test_y, scaler_y = ml_scaler.get_scaled_test_data(data_y, y_scaler_file_path, scaler_param)
    return test_X, scaler_X , test_y, scaler_y 

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


#### 1-2-2. Random Nan Insert (Test X)
#### 1-2-3. Data scaling

In [12]:
split_mode = model_meta['dataSplitMode']
transformParameter = model_meta["transformParameter"]
test_X_array, test_y_array = ML_pipeline.transform_data_by_split_mode(split_mode, transformParameter, test_X, test_y)

nan_limit_num:  12
Original num: 2005 Final num: 1979 NaN num: 26


In [13]:
modelParameter = model_meta["modelParameter"]

testParameter = {
    'device': 'cpu',
    'batch_size': model_meta["trainParameter"]['batch_size']
}

In [14]:
#3. Test model
preds, trues = ML_pipeline.CLUST_regresstion_test(test_X_array, test_y_array, testParameter, model_method, model_file_path, modelParameter)

ModuleNotFoundError: No module named 'Clust.clust.ML.regression_YK.models.rnn_model'

In [ ]:
print(test_X_array.shape, test_y_array.shape)
print(preds.shape, trues.shape)
target = feature_y_list[0]
scaler, feature_list = get_scaler_information_by_y_flag(data_y_flag, scaler_X, scaler_y, feature_X_list, feature_y_list)
df_result, result_metrics = get_final_metrics(preds, trues, scaler_param, scaler, feature_list, target)

In [ ]:
print(df_result)
print(result_metrics)
df_result.plot()

In [ ]:
# 5. Only For IPython Notebook
#from Clust.clust.tool.plot.plot_two_data import plot_predictions
#import plotly.offline as pyo
# Set notebook mode to work in offline
#pyo.init_notebook_mode()
#plot_predictions(df_result)